In [2]:
import numpy as np
import cvxpy as cp

In [38]:
from matplotlib import pyplot as plt

### Examples of cvxpy and cvxpylayers

In [4]:
m = 15
n = 10
np.random.seed(1)
s0 = np.random.randn(m)
lamb0 = np.maximum(-s0, 0)
s0 = np.maximum(s0, 0)
x0 = np.random.randn(n)
A = np.random.randn(m, n)
b = A @ x0 + s0
c = -A.T @ lamb0

In [5]:
# Define and solve the CVXPY problem.
x = cp.Variable(n)
objective = cp.Minimize(c.T@x + cp.sum_squares(x) * 1e-2)
constraints = [
    A @ x <= b,
    cp.sum_squares(A @ x) <= np.sum(b ** 2) / 2
]
prob = cp.Problem(objective, constraints)
prob.solve()

-13.388855525388324

In [6]:
# Print result.
print("\nThe optimal value is", prob.value)
print("A solution x is")
print(x.value)
print("A dual solution is")
print(prob.constraints[0].dual_value)


The optimal value is -13.388855525388324
A solution x is
[-1.21425975 -0.22544726 -0.82587237 -0.4940789   0.13023043 -0.46215896
  1.4850115   0.42419635  1.19464608  1.87527732]
A dual solution is
[1.12011945e-08 1.19282824e+00 2.13491766e+00 5.43570788e-08
 2.21013673e-08 1.39783152e+00 2.70849582e-08 5.31067943e-08
 2.05629661e-08 1.74569441e-08 6.16643520e-01 2.85555214e-04
 2.98221760e-01 2.87084912e-01 1.24290736e-08]


In [7]:
n, m = 2, 3
x = cp.Variable(n)
A = cp.Parameter((m, n))
b = cp.Parameter(m)
constraints = [x + cp.sum_squares(x) <= 0]
objective = cp.Minimize(0.5 * cp.pnorm(A @ x - b, p=1))
problem = cp.Problem(objective, constraints)
assert problem.is_dpp()

In [8]:
import cvxpy as cp
import torch
from cvxpylayers.torch import CvxpyLayer

n, m = 2, 3
x = cp.Variable(n)
A = cp.Parameter((m, n))
b = cp.Parameter(m)
constraints = [x >= 0]
objective = cp.Minimize(0.5 * cp.pnorm(A @ x - b, p=1))
problem = cp.Problem(objective, constraints)
assert problem.is_dpp()

cvxpylayer = CvxpyLayer(problem, parameters=[A, b], variables=[x])
A_tch = torch.randn(m, n, requires_grad=True)
b_tch = torch.randn(m, requires_grad=True)

# solve the problem
solution, = cvxpylayer(A_tch, b_tch)

# compute the gradient of the sum of the solution with respect to A, b
solution.sum().backward()

### Synthetic Data from Kallus 2018

In [9]:
np.random.rand(10)

array([0.43367635, 0.80736053, 0.3152448 , 0.89288871, 0.57785722,
       0.1840102 , 0.78792923, 0.61203118, 0.05390927, 0.42019368])

In [53]:
beta0 = 2.5
beta0_t = -2
beta_x = np.asarray([0, .5, -0.5, 0, 0])
beta_x_t = np.asarray([-1.5, 1, -1.5, 1., 0.5])
beta_xi = 1
beta_xi_t = -2
beta_e_x = np.asarray([0, .75, -.5, 0, -1])
mu_x = np.asarray([-1, .5, -1, 0, -1]);

def generate_data(n):
    xi = (np.random.rand(n) > 0.5).astype(int)
    X = mu_x[None, :] + np.random.randn(n * 5).reshape(n, 5)
    eps = [np.random.randn(n) for t in (0, 1)]
    Y = np.array([
        X @ (beta_x + beta_x_t * t) + (beta_xi + beta_xi_t * t) * xi + (beta0 + beta0_t * t) + eps[t]
        for t in (0, 1)
    ])
    U = (Y[0, :] > Y[1, :]).astype(int)
    z = X @ beta_e_x
    e_x = np.exp(z) / (1 + np.exp(z))
    e_xu = (6 * e_x) / (4 + 5 * U + e_x * (2 - 5 * U))
    T = (np.random.rand(n) < e_xu).astype(int)
    Y = Y[T, range(n)]
    return Y, T, X, U, e_x, e_xu

def evaluate_policy(policy, n=1000):
    xi = (np.random.rand(n) > 0.5).astype(int)
    X = mu_x[None, :] + np.random.randn(n * 5).reshape(n, 5)
    eps = [np.random.randn(n) for t in (0, 1)]
    Y = np.array([
        X @ (beta_x + beta_x_t * t) + (beta_xi + beta_xi_t * t) * xi + (beta0 + beta0_t * t) + eps[t]
        for t in (0, 1)
    ])
    e_x = policy(X)
    T = (np.random.rand(n) < e_x).astype(int)
    Y = Y[T, range(n)]
    return Y.mean()

In [258]:
Y, T, X, U, e_x, e_xu = generate_data(800)

In [259]:
def toy_policy(X):
    z = X @ beta_e_x
    e_x = np.exp(z) / (1 + np.exp(z))
    return e_x

def zero_policy(X):
    return np.zeros(X.shape[0], dtype=int)

evaluate_policy(toy_policy)

3.9177610212929896

### Implement Confounding Robust Hajek Estimator    

In [231]:
def confoundingRobustHajek(r, T, a, b, return_w=False):
    r_np = r.data.numpy() if isinstance(r, torch.Tensor) else r
    w = np.zeros_like(a)
    ret = torch.zeros(1)
    for t in set(T):
        r_t = r_np[T == t]
        a_t = a[T == t]
        b_t = b[T == t]
        w[T == t] = hajekLP(r_t, a_t, b_t)
    if isinstance(r, torch.Tensor):
        est = torch.mean(torch.as_tensor(w) * r)
    else:
        est = np.mean(w * r_np)
    return (est, w) if return_w else est

def hajekLP(r, a, b):
    n = r.shape[0]
    w = cp.Variable(n)
    phi = cp.Variable(1)
    constraints = [
        np.zeros(n) <= w,
        0 <= phi,
        a * phi <= w,
        w <= b * phi,
        cp.sum(w) == n
    ]
    objective = cp.Maximize(cp.sum(r * w))
    problem = cp.Problem(objective, constraints)
    problem.solve()
    return w.value / phi.value

In [232]:
def get_a_b(e_x, gamma):
    a = 1 + 1 / gamma * (1 / e_x - 1)
    b = 1 + gamma * (1 / e_x - 1)
    return a, b

In [253]:
a, b = get_a_b(e_x, 1.5)
est = confoundingRobustHajek(Y * toy_policy(X), T, a, b)

/home/kei/Desktop/env_pillbox/lib/python3.8/site-packages/cvxpy/expressions/expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 86 times so far.

  warnings.warn(msg, UserWarning)
/home/kei/Desktop/env_pillbox/lib/python3.8/site-packages/cvxpy/expressions/expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 87 times so far.

  warnings.warn(msg, U

In [254]:
est

4.05609094141064

In [255]:
differentiable_r = torch.tensor(np.random.randn(n), requires_grad=True)
est, w = confoundingRobustHajek(differentiable_r, T, a, b, return_w=True)

/home/kei/Desktop/env_pillbox/lib/python3.8/site-packages/cvxpy/expressions/expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 88 times so far.

  warnings.warn(msg, UserWarning)
/home/kei/Desktop/env_pillbox/lib/python3.8/site-packages/cvxpy/expressions/expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 89 times so far.

  warnings.warn(msg, U

In [256]:
est.backward()

In [257]:
torch.allclose(differentiable_r.grad, torch.as_tensor(w / n))

True

### Implement Confounding Robust Kernel Estimator
- This requires consistent nominal propensity score (p_obs(t|x))
- For discrete t, just run kernel logistic regression

In [273]:
from sklearn.decomposition import KernelPCA
from sklearn.linear_model import LogisticRegressionCV

Z = KernelPCA(n_components=400).fit_transform(X)
clf = LogisticRegressionCV().fit(Z, T)
e_xx = clf.predict_proba(Z)[:, 1]

np.mean(np.abs(e_xx - e_x))

0.039824902382792536

In [ ]:
import optuna
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

def objective(

### Compare Policy Improvement Performance